In [1]:
# 구글 드라이브 접근
# from google.colab import drive as gd
# gd.mount( '/content/gdrive' )

# data 파일 정의
DefaultRootFolder = 'data'
SourceDataFolder = DefaultRootFolder + '/'
TrainDataFile = SourceDataFolder + 'train.csv'
TestDataFile = SourceDataFolder + 'test.csv'
SubmissingFile = SourceDataFolder + 'sample_submission.csv'

WorkFolder = 'imsee/'
WorkTrainData = WorkFolder + '/sourcedata/Save_x_train.csv' 
WorkTrainResultData = WorkFolder + '/sourcedata/Save_y_train.csv' 
WorkTestData = WorkFolder + '/sourcedata/Save_x_test.csv' 
OutputSubmissionFile = WorkFolder + '/submission.csv'

OutputTrainDataTemplate = SourceDataFolder + 'TempTrainData.csv'

In [ ]:
# !pip install bayesian-optimization

In [2]:
!pip install xgboost

In [3]:
import pandas as pd                         # 데이터 분석 라이브러리
import numpy as np                          # 계산 라이브러리
from tqdm import tqdm                       # 진행바
from sklearn.metrics import roc_auc_score   # AUC 스코어 계산
import xgboost as xgb
from sklearn.model_selection import KFold   # K-fold CV
from bayes_opt import BayesianOptimization  # 베이지안 최적화 라이브러리
from functools import partial               # 함수 변수 고정
import warnings                             
warnings.filterwarnings("ignore")           # 경고 문구 미표시
import random                               # random 변수 생성
import re                                   # 정규식 처리

In [4]:
train = pd.read_csv( TrainDataFile )
# display( train )

In [5]:
# data 확인 용
winners = train.groupby(['game_id']).winner.max()
display( winners[38871] )

0

In [6]:
def species_converter(string):
    if string == 'T':
        return 0
    elif string == 'P':
        return 1
    elif string == 'Z':
        return 2
    else:
        raise ValueError
        
def data_processing( train_data, answer=False ) :
  # 불필요한 data의 삭제
  #aa = train_data[ train_data.event_contents.isin( ['[]'] ) ].index
  #train_data.drop( aa, inplace=True)

  # game_id의 unique한 목록
  game_ids = train_data['game_id'].unique()
  # 발생되는 event
  events = train_data.event.unique()

  # Ability 분석
  p1 = re.compile( '\([^)]*\)\ \-\ ' )    # "( xxx ) - "형식의 문자열  
  p2 = re.compile( '; Location:[^\)]*\)' )  # "; Location: ..."  형식의 문자열
  p3 = re.compile( '; Target:\ [\w]*', re.DOTALL) # "; Target: ..." 형식의 문자열
  p4 = re.compile( '\ \[[\w]*\]' ) # " [xxxx]" 형식의 문자열
  p5 = re.compile( '\([\w]*\)' ) # "(xxxx)" 형식의 문자열
  train_data.event_contents.fillna( '' )
  train_data['new_event_contents'] = [p5.sub('', p4.sub('', p3.sub('', p2.sub( '', p1.sub( '', str( v ) ) ) ) ) ) for v in tqdm(train_data.event_contents)] 
  train_data.new_event_contents.fillna( '' )
  event_contents_unique = train_data.new_event_contents[train_data.event.isin( ['Ability'] )].unique()

  #Right Click 분석
  p1 = re.compile( ':\ \([^)]*\)' ) # ": ..."  형식의 문자열
  p2 = re.compile( '\ \[[^)]*\]' )  # " [03BC0001]"형식의 문자열
  pend = re.compile( '[\:\;\ ]*' ) # ":", ";", " "
  train_data['new_rightclick_contents'] = [pend.sub('', p2.sub('', p1.sub('', str( v ) ) ) ) for v in tqdm(train_data.event_contents)] 
  train_data.new_rightclick_contents.fillna( '' )
  event_contents_rightclick_group = train_data.new_rightclick_contents[train_data.event.isin( ['Right Click'] )].unique()

  # 초기화용 array
  unique_event_0, unique_event_1, delta_event = {}, {}, {}
  #norm_mu_event_0, norm_sigma_event_0, norm_mu_event_1, norm_sigma_event_1  = {}, {}, {}, {}
  for event in events:
    unique_event_0['P0_' + event] = 0
    unique_event_1['P1_' + event] = 0
    delta_event['delta_' + event] = 0
    '''
    norm_mu_event_0['P0_norm_' + event] = 0
    norm_sigma_event_0['P0_norm_' + event] = 0
    norm_mu_event_1['P1_norm_' + event] = 0
    norm_sigma_event_1['P1_norm_' + event] = 0
    '''

  ability_event_0, ability_event_1, ability_delta_event = {}, {}, {}
  for ability in event_contents_unique : 
    ability_event_0['Ability0_'+ability] = 0
    ability_event_1['Ability1_'+ability] = 0
    ability_delta_event['Ability_delta_'+ability] = 0

  rc_event_0, rc_event_1, rc_delta_event = {}, {}, {}
  for rc in event_contents_rightclick_group : 
    rc_event_0['rc0_'+rc] = 0
    rc_event_1['rc1_'+rc] = 0
    rc_delta_event['rc_delta_'+rc] = 0    
  print('변수 초기화 완료')

  # game_id, player별 종족 목록, 
  # 1번 game에 1번 player, 저그 이런식으로 
  species = train_data.groupby(['game_id', 'player']).species.unique()
  # game_id, player별 event별 count값
  event_count = train_data.groupby(['game_id', 'player']).event.value_counts()

  # event가 Ability인 data중 game_id, player별, event로  new_event_contents별 count를 계산 한다.
  ability_count = train_data[ train_data['event'].isin(['Ability']) ].groupby(['game_id', 'player','event']).new_event_contents.value_counts()
  ability_count = pd.DataFrame( ability_count )

  # event가 Right Click인 data중 game_id, player별, event로  new_event_contents별 count를 계산 한다.
  rightclick_count = train_data[ train_data['event'].isin(['Right Click']) ].groupby(['game_id', 'player','event']).new_rightclick_contents.value_counts()
  rightclick_count = pd.DataFrame( rightclick_count )
  print('event별 count값 집계')
  '''
  timedata = train_data.groupby(['game_id', 'player', 'event']).time.value_counts()
  timedata = pd.DataFrame( timedata )
  display(timedata)
  print('time data 집계')
  '''

  if answer :
    # 각 game의 승리자 목록
    winners = train_data.groupby(['game_id']).winner.max()

  x_data, y_data = [], []

  for game_id in  tqdm( game_ids) :
    # game_id에 대한 player별 event count를 계산 한다.
    df_event_count = event_count[game_id].unstack(level=-1) 
    # game_id에 대한 player의 종족
    df = pd.DataFrame( species[game_id] )
    # game_id에 대한 player별 종족과 event별 count값, event가 없는 값은 NaN으로 출력 된다.
    df = pd.concat( [df, df_event_count], axis=1 )
    df = df.fillna( 0 ) # NaN값을 0으로 채운다

    # player별 종족을 dataframe으로 만든다.
    df_P0_species = pd.DataFrame([species_converter(df.loc[0]['species'][0])], columns=['P0_species'])        
    df_P1_species = pd.DataFrame([species_converter(df.loc[1]['species'][0])], columns=['P1_species'])
    # df에 저장된 종족 field를 삭제 한다.
    df.drop(['species'], axis=1, inplace=True)

    # player0에 대한 event별 count를 누적 시킨다.
    df_P0_event = unique_event_0.copy() # array를 0으로 초기화 시킨다.
    for column in df.columns :
      df_P0_event['P0_' + column] = df.loc[0][column] 
    df_P0_event = pd.DataFrame(pd.Series(df_P0_event)).T

    # player1에 대한 event별 count를 누적 시킨다.
    df_P1_event = unique_event_1.copy() # array를 0으로 초기화 시킨다.
    for column in df.columns :
      df_P1_event['P1_' + column] = df.loc[1][column] 
    df_P1_event = pd.DataFrame(pd.Series(df_P1_event)).T
    
    # player0과 player1과의 event별 count 차이를 계산 한다.
    df_delta_event = delta_event.copy()
    for column in df.columns :
      df_delta_event['delta_'+column] = df_P0_event['P0_' + column][0] - df_P1_event['P1_' + column][0]
    df_delta_event = pd.DataFrame(pd.Series(df_delta_event)).T

    # ability에 따른 count 차이를 계산
    df_ability_0 = ability_event_0.copy()
    df_ability_1 = ability_event_1.copy()
    df_ability_delta = ability_delta_event.copy()
    for ability in event_contents_unique :
      try :
        df_ability_0['Ability0_'+ability] = ability_count.loc[game_id, 0, 'Ability', ability][0]
      except KeyError :
        df_ability_0['Ability0_'+ability] = 0
      try :
        df_ability_1['Ability1_'+ability] = ability_count.loc[game_id, 1, 'Ability', ability][0]
      except KeyError :
        df_ability_1['Ability1_'+ability] = 0

    df_ability_0 = pd.DataFrame(pd.Series(df_ability_0)).T
    df_ability_1 = pd.DataFrame(pd.Series(df_ability_1)).T
    for ability in event_contents_unique :
      df_ability_delta['Ability_delta_'+ability] = df_ability_0['Ability0_'+ability][0] - df_ability_1['Ability1_'+ability][0]
    df_ability_delta = pd.DataFrame(pd.Series(df_ability_delta)).T

    # right click 에 따른 count 차이를 계산
    df_rc_0 = rc_event_0.copy()
    df_rc_1 = rc_event_1.copy()
    df_rc_delta = rc_delta_event.copy()
    for rc in event_contents_rightclick_group :
      try :
        df_rc_0['rc0_'+rc] = rightclick_count.loc[game_id, 0, 'Right Click', rc][0]
      except KeyError :
        df_rc_0['rc0_'+rc] = 0
      try:
        df_rc_1['rc1_'+rc] = rightclick_count.loc[game_id, 1, 'Right Click', rc][0]
      except KeyError :
        df_rc_1['rc1_'+rc] = 0
    df_rc_0 = pd.DataFrame(pd.Series(df_rc_0)).T
    df_rc_1 = pd.DataFrame(pd.Series(df_rc_1)).T
    for rc in event_contents_rightclick_group :
      df_rc_delta['rc_delta_'+rc] = df_rc_0['rc0_'+rc][0] - df_rc_1['rc1_'+rc][0]
    df_rc_delta = pd.DataFrame(pd.Series(df_rc_delta)).T

    #out = pd.concat([df_P0_species, df_P0_event, df_P1_species, df_P1_event, df_delta_event], axis=1)
    #out = pd.concat([df_P0_species, df_P0_event, df_P0_norm_mu_event, df_P0_norm_sigma_event, df_P1_species, df_P1_event, df_P0_norm_mu_event, df_P0_norm_sigma_event, df_delta_event], axis=1) 확률 분포 포함
    out = pd.concat([df_P0_species, df_P0_event, df_ability_0, df_rc_0, df_P1_species, df_P1_event, df_ability_1, df_rc_1, df_delta_event, df_ability_delta, df_rc_delta], axis=1)
    out.index = [game_id]
    out.index.name = 'game_id'

    x_data.append(out)
    if answer :
      y_data.append(winners[game_id])

  x_data = pd.concat(x_data) # game별 player별 종족, event별 count
  y_data = np.array(y_data)  # 승리자 list

  return x_data, y_data

In [7]:
# train data를 가공 처리 한다.
x_train, y_train = data_processing(train, True )
display( x_train )
x_train.to_csv(WorkTrainData)
display( y_train )
pd.DataFrame( y_train ).to_csv(WorkTrainResultData)

100%|██████████████████████████████████████████████████████████████████| 67091776/67091776 [04:53<00:00, 228762.13it/s]


변수 초기화 완료
event별 count값 집계


100%|██████████████████████████████████████████████████████████████████████████| 38872/38872 [1:04:43<00:00, 10.01it/s]


,P0_species,P0_Camera,P0_Selection,P0_Ability,P0_Right Click,P0_SetControlGroup,P0_GetControlGroup,P0_AddToControlGroup,P0_ControlGroup,Ability0_TrainSCV,...,rc_delta_TargetInfestorLocation,rc_delta_TargetUltraliskLocation,rc_delta_TargetBroodLordWeaponLocation,rc_delta_TargetBeacon_TerranLocation,rc_delta_TargetBroodLordLocation,rc_delta_TargetBeacon_ZergLocation,rc_delta_TargetDisruptorLocation,rc_delta_TargetExtendingBridgeNEWide8OutLocation,rc_delta_TargetInfestorBurrowedLocation,rc_delta_TargetHydraliskBurrowedLocation
game_id,,,,,,,,,,,,,,,,,,,,,
0,0,444.0,50.0,34.0,35.0,3.0,24.0,2.0,0.0,9,...,0,0,0,0,0,0,0,0,0,0
1,1,627.0,186.0,77.0,160.0,10.0,162.0,1.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,413.0,90.0,69.0,160.0,14.0,99.0,6.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,713.0,180.0,82.0,276.0,6.0,132.0,0.0,0.0,7,...,0,0,0,0,0,0,0,0,0,0
4,0,430.0,67.0,57.0,177.0,10.0,224.0,1.0,0.0,14,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38867,0,639.0,53.0,43.0,95.0,1.0,0.0,0.0,0.0,11,...,0,0,0,0,0,0,0,0,0,0
38868,0,558.0,62.0,63.0,123.0,0.0,0.0,0.0,0.0,7,...,0,0,0,0,0,0,0,0,0,0
38869,1,785.0,106.0,56.0,248.0,4.0,28.0,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0


array([1, 1, 0, ..., 0, 1, 0], dtype=int64)

In [4]:
# Error가 나면 임시 파일을 읽는다
x_train = pd.read_csv(WorkTrainData, index_col='game_id') # ok
y_train = pd.read_csv(WorkTrainResultData, names=['c1', 'c2'], header=0)
y_train = np.array( y_train.c2 )

In [5]:
x_test = pd.read_csv(WorkTestData, index_col='game_id') # ok

In [8]:
# 검증 data 읽기
test = pd.read_csv(TestDataFile)
display( test )
# 검증 data 가공
x_test, _ = data_processing(test, answer=False)
display(x_test)
x_test.to_csv(WorkTestData)

,game_id,time,player,species,event,event_contents
0,38872,0.00,0,P,Camera,"at (22.25, 81.5078125)"
1,38872,0.00,1,P,Camera,"at (120.25, 153.83984375)"
2,38872,0.01,1,P,Selection,['Nexus [3100001]']
3,38872,0.01,1,P,Ability,(15E0) - TrainProbe
4,38872,0.01,1,P,AddToControlGroup,NaN
...,...,...,...,...,...,...
28714844,55658,4.54,1,T,Right Click,"Location: (120.584228515625, 56.930419921875, ..."
28714845,55658,4.54,0,Z,Camera,"at (70.9921875, 117.65234375)"
28714846,55658,4.54,1,T,Right Click,"Location: (122.146728515625, 55.52099609375, 3..."
28714847,55658,4.54,1,T,Right Click,"Location: (123.6767578125, 54.140625, 33215)"


100%|██████████████████████████████████████████████████████████████████| 28714849/28714849 [02:03<00:00, 233361.75it/s]


변수 초기화 완료


  0%|                                                                                | 1/16787 [00:00<50:22,  5.55it/s]

event별 count값 집계


100%|████████████████████████████████████████████████████████████████████████████| 16787/16787 [26:36<00:00, 10.51it/s]


,P0_species,P0_Camera,P0_Selection,P0_Ability,P0_AddToControlGroup,P0_SetControlGroup,P0_Right Click,P0_GetControlGroup,P0_ControlGroup,Ability0_TrainProbe,...,rc_delta_TargetChangelingLocation,rc_delta_TargetExtendingBridgeNEWide8OutLocation,rc_delta_TargetBroodLordAWeaponLocation,rc_delta_TargetExtendingBridgeNEWide10OutLocation,rc_delta_TargetFusionCoreLocation,rc_delta_TargetBanelingBurrowedLocation,rc_delta_TargetCorruptorLocation,rc_delta_TargetGhostAcademyLocation,rc_delta_TargetDisruptorLocation,rc_delta_TargetChangelingMarineShieldLocation
game_id,,,,,,,,,,,,,,,,,,,,,
38872,1,232.0,52.0,31.0,1.0,5.0,241.0,9.0,0.0,11,...,0,0,0,0,0,0,0,0,0,0
38873,2,200.0,114.0,34.0,0.0,0.0,51.0,18.0,4.0,0,...,0,0,0,0,0,0,0,0,0,0
38874,0,245.0,42.0,33.0,0.0,7.0,79.0,63.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
38875,0,515.0,123.0,95.0,8.0,9.0,218.0,255.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
38876,2,129.0,25.0,7.0,2.0,2.0,30.0,387.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55654,2,258.0,186.0,37.0,2.0,6.0,136.0,693.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
55655,2,810.0,317.0,62.0,0.0,7.0,158.0,217.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
55656,1,330.0,67.0,14.0,0.0,1.0,106.0,0.0,0.0,6,...,0,0,0,0,0,0,0,0,0,0


In [6]:
# 컬럼 불일치 여부 테스트 코드들
diff_columns = [x for x in x_train.columns.unique() if x not in x_test.columns.unique()]

In [7]:
diff_columns_2 = [ y for y in x_test.columns.unique() if y not in x_train.columns.unique()]

In [8]:
x_train.drop(columns=diff_columns, inplace=True)

In [9]:
x_test.drop(columns=diff_columns_2, inplace=True)

In [10]:
x_train.shape

(38872, 1538)

In [11]:
x_test.shape

(16787, 1538)

In [13]:
train_columns = list(x_train.columns.unique())

In [14]:
x_test = x_test[train_columns] # 컬럼 맞추기

In [16]:
x_train

,P0_species,P0_Camera,P0_Selection,P0_Ability,P0_Right Click,P0_SetControlGroup,P0_GetControlGroup,P0_AddToControlGroup,P0_ControlGroup,Ability0_TrainSCV,...,rc_delta_TargetBroodLordAWeaponLocation,rc_delta_TargetQueenBurrowedLocation,rc_delta_TargetInfestorLocation,rc_delta_TargetUltraliskLocation,rc_delta_TargetBroodLordWeaponLocation,rc_delta_TargetBeacon_TerranLocation,rc_delta_TargetBroodLordLocation,rc_delta_TargetDisruptorLocation,rc_delta_TargetExtendingBridgeNEWide8OutLocation,rc_delta_TargetInfestorBurrowedLocation
game_id,,,,,,,,,,,,,,,,,,,,,
0,0,444.0,50.0,34.0,35.0,3.0,24.0,2.0,0.0,9,...,0,0,0,0,0,0,0,0,0,0
1,1,627.0,186.0,77.0,160.0,10.0,162.0,1.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,413.0,90.0,69.0,160.0,14.0,99.0,6.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,713.0,180.0,82.0,276.0,6.0,132.0,0.0,0.0,7,...,0,0,0,0,0,0,0,0,0,0
4,0,430.0,67.0,57.0,177.0,10.0,224.0,1.0,0.0,14,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38867,0,639.0,53.0,43.0,95.0,1.0,0.0,0.0,0.0,11,...,0,0,0,0,0,0,0,0,0,0
38868,0,558.0,62.0,63.0,123.0,0.0,0.0,0.0,0.0,7,...,0,0,0,0,0,0,0,0,0,0
38869,1,785.0,106.0,56.0,248.0,4.0,28.0,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
x_test

,P0_species,P0_Camera,P0_Selection,P0_Ability,P0_Right Click,P0_SetControlGroup,P0_GetControlGroup,P0_AddToControlGroup,P0_ControlGroup,Ability0_TrainSCV,...,rc_delta_TargetBroodLordAWeaponLocation,rc_delta_TargetQueenBurrowedLocation,rc_delta_TargetInfestorLocation,rc_delta_TargetUltraliskLocation,rc_delta_TargetBroodLordWeaponLocation,rc_delta_TargetBeacon_TerranLocation,rc_delta_TargetBroodLordLocation,rc_delta_TargetDisruptorLocation,rc_delta_TargetExtendingBridgeNEWide8OutLocation,rc_delta_TargetInfestorBurrowedLocation
game_id,,,,,,,,,,,,,,,,,,,,,
38872,1,232.0,52.0,31.0,241.0,5.0,9.0,1.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
38873,2,200.0,114.0,34.0,51.0,0.0,18.0,0.0,4.0,0,...,0,0,0,0,0,0,0,0,0,0
38874,0,245.0,42.0,33.0,79.0,7.0,63.0,0.0,0.0,9,...,0,0,0,0,0,0,0,0,0,0
38875,0,515.0,123.0,95.0,218.0,9.0,255.0,8.0,0.0,21,...,0,0,0,0,0,0,0,0,0,0
38876,2,129.0,25.0,7.0,30.0,2.0,387.0,2.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55654,2,258.0,186.0,37.0,136.0,6.0,693.0,2.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
55655,2,810.0,317.0,62.0,158.0,7.0,217.0,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
55656,1,330.0,67.0,14.0,106.0,1.0,0.0,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
# XGBClassifier
# 반드시 튜닝해야할 파라미터는 max_depth / gamma

def XGB_cv(max_depth, learning_rate, subsample, colsample_bytree, reg_alpha, reg_lambda, gamma, max_delta_step, x_data=None, y_data=None, n_splits=5, output='score'):
    score = 0
    kf = KFold(n_splits = n_splits)
    models = []
    for train_index, valid_index in kf.split(x_data):
        x_train, y_train = x_data.iloc[train_index], y_data[train_index]
        x_valid, y_valid = x_data.iloc[valid_index], y_data[valid_index]

        model = xgb.XGBClassifier(
            # booster = 'dart',
            max_depth = int(max_depth),
            # n_estimators = int(n_estimators),
            learning_rate = learning_rate,
            subsample = np.clip(subsample, 0, 1),
            # colsample_bynode = np.clip(colsample_bynode, 0, 1),
            colsample_bytree = np.clip(colsample_bytree, 0, 1),
            reg_alpha = reg_alpha,
            reg_lambda = reg_lambda,
            gamma = gamma,
            max_delta_step = max_delta_step,
        )

        model.fit(x_train, y_train)
        models.append(model)

        pred = model.predict_proba(x_valid)[:, 1]
        true = y_valid
        score += roc_auc_score(true, pred) / n_splits

    if output == 'score':
        return score
    if output == 'model':
        return models

In [16]:
# 모델 그 외 변수는 고정
var_fixed = partial(XGB_cv, x_data=x_train, y_data=y_train, n_splits=5, output='score')
# 베이지안 최적화 범위 설정
XGBo = BayesianOptimization(
    var_fixed, 
    {
        'max_depth': (8, 512), # ~ 500
        'learning_rate': (0.01, 0.6), #0.6xxx
        'subsample': (0.5, 0.999), # 0.5 ~ 0.7 / 0.2 ?
        'colsample_bytree': (0.5, 1), # 0.5 ~ 0.8
        # 'colsample_bynode':(0, 1),
        # 'n_estimators' : (16, 1024),
        'reg_alpha' : (3, 40), # L1 - 7 ~ 30
        'reg_lambda' : (5, 70), # L2 - 8 ~ 45
        'max_delta_step' : (1, 25), #8.x
        'gamma' : (1, 10), # 1 ~ 10 clipping 잘 안됨.
    },
    random_state = 4321 # set seed 4321 / random.randrange(50000)
)
XGBo.maximize(init_points=15, n_iter=60) # 처음 15회 랜덤값으로 score 계산 후 60회 최적화

|   iter    |  target   | colsam... |   gamma   | learni... | max_de... | max_depth | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------------------
|  1        |  0.653    |  0.5354   |  13.23    |  0.4631   |  7.873    |  105.3    |  39.22    |  31.4     |  0.8031   |
|  2        |  0.6812   |  0.5446   |  5.648    |  0.3752   |  12.04    |  118.0    |  27.55    |  49.11    |  0.8801   |
|  3        |  0.6533   |  0.6406   |  10.3     |  0.2361   |  10.61    |  483.1    |  37.41    |  66.64    |  0.6502   |
|  4        |  0.6616   |  0.6711   |  10.97    |  0.03497  |  6.574    |  224.8    |  5.884    |  54.83    |  0.8416   |
|  5        |  0.6882   |  0.5752   |  2.517    |  0.1698   |  1.724    |  430.3    |  25.11    |  65.52    |  0.6982   |
|  6        |  0.6844   |  0.6882   |  4.657    |  0.3651   |  8.913    |  240.7    |  29.82    |  57.77    |  0.8944   |
|  7        |  0.6484   

|  41       |  0.6829   |  0.8638   |  2.096    |  0.2096   |  22.39    |  85.71    |  4.139    |  10.33    |  0.7881   |
|  42       |  0.6693   |  0.5884   |  1.569    |  0.2876   |  24.78    |  485.0    |  3.999    |  18.12    |  0.6274   |
|  43       |  0.6735   |  0.5893   |  5.637    |  0.5487   |  1.784    |  10.39    |  37.37    |  5.167    |  0.7448   |
|  44       |  0.6793   |  0.5618   |  2.32     |  0.3594   |  1.423    |  306.0    |  3.244    |  68.15    |  0.7297   |
|  45       |  0.6888   |  0.6611   |  13.2     |  0.1915   |  1.287    |  322.7    |  4.301    |  6.493    |  0.8171   |
|  46       |  0.6597   |  0.6046   |  8.276    |  0.0865   |  24.47    |  303.8    |  38.69    |  27.87    |  0.8038   |
|  47       |  0.6559   |  0.8351   |  10.69    |  0.08701  |  24.95    |  507.7    |  39.81    |  7.226    |  0.8414   |
|  48       |  0.6759   |  0.5546   |  1.822    |  0.4299   |  23.65    |  148.8    |  4.188    |  63.0     |  0.8874   |
|  49       |  0.6732   

In [17]:
params = XGBo.max['params']
models = XGB_cv(
    params['max_depth'], 
    params['learning_rate'],
    params['subsample'], 
    params['colsample_bytree'],
    # params['colsample_bynode'], 
    params['reg_alpha'], 
    params['reg_lambda'],
    params['max_delta_step'],
    # params['n_estimators'],
    params['gamma'],
    x_data=x_train, y_data=y_train, n_splits=5, output='model')

In [ ]:
# print([z for z in x_test.columns.unique() if z not in x_train.columns.unique()])

In [ ]:
# print([a for a in x_train.columns.unique() if a not in x_test.columns.unique()])

In [ ]:
# print(x_train.columns.unique() != x_test.columns.unique())

In [19]:
x_test.shape

(16787, 1538)

In [20]:
x_train.shape

(38872, 1538)

In [21]:
y_train.shape

(38872,)

In [22]:
x_train.head()

,P0_species,P0_Camera,P0_Selection,P0_Ability,P0_Right Click,P0_SetControlGroup,P0_GetControlGroup,P0_AddToControlGroup,P0_ControlGroup,Ability0_TrainSCV,...,rc_delta_TargetBroodLordAWeaponLocation,rc_delta_TargetQueenBurrowedLocation,rc_delta_TargetInfestorLocation,rc_delta_TargetUltraliskLocation,rc_delta_TargetBroodLordWeaponLocation,rc_delta_TargetBeacon_TerranLocation,rc_delta_TargetBroodLordLocation,rc_delta_TargetDisruptorLocation,rc_delta_TargetExtendingBridgeNEWide8OutLocation,rc_delta_TargetInfestorBurrowedLocation
game_id,,,,,,,,,,,,,,,,,,,,,
0,0,444.0,50.0,34.0,35.0,3.0,24.0,2.0,0.0,9,...,0,0,0,0,0,0,0,0,0,0
1,1,627.0,186.0,77.0,160.0,10.0,162.0,1.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,413.0,90.0,69.0,160.0,14.0,99.0,6.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,713.0,180.0,82.0,276.0,6.0,132.0,0.0,0.0,7,...,0,0,0,0,0,0,0,0,0,0
4,0,430.0,67.0,57.0,177.0,10.0,224.0,1.0,0.0,14,...,0,0,0,0,0,0,0,0,0,0


In [28]:
x_test.head()

,P0_species,P0_Camera,P0_Selection,P0_Ability,P0_AddToControlGroup,P0_SetControlGroup,P0_Right Click,P0_GetControlGroup,P0_ControlGroup,Ability0_TrainProbe,...,rc_delta_TargetChangelingLocation,rc_delta_TargetExtendingBridgeNEWide8OutLocation,rc_delta_TargetBroodLordAWeaponLocation,rc_delta_TargetExtendingBridgeNEWide10OutLocation,rc_delta_TargetFusionCoreLocation,rc_delta_TargetBanelingBurrowedLocation,rc_delta_TargetCorruptorLocation,rc_delta_TargetGhostAcademyLocation,rc_delta_TargetDisruptorLocation,rc_delta_TargetChangelingMarineShieldLocation
game_id,,,,,,,,,,,,,,,,,,,,,
38872,1,232.0,52.0,31.0,1.0,5.0,241.0,9.0,0.0,11,...,0,0,0,0,0,0,0,0,0,0
38873,2,200.0,114.0,34.0,0.0,0.0,51.0,18.0,4.0,0,...,0,0,0,0,0,0,0,0,0,0
38874,0,245.0,42.0,33.0,0.0,7.0,79.0,63.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
38875,0,515.0,123.0,95.0,8.0,9.0,218.0,255.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
38876,2,129.0,25.0,7.0,2.0,2.0,30.0,387.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
train_columns

['P0_species',
 'P0_Camera',
 'P0_Selection',
 'P0_Ability',
 'P0_Right Click',
 'P0_SetControlGroup',
 'P0_GetControlGroup',
 'P0_AddToControlGroup',
 'P0_ControlGroup',
 'Ability0_TrainSCV',
 'Ability0_BuildSupplyDepot',
 'Ability0_Stop',
 'Ability0_BuildRefinery',
 'Ability0_BuildBarracks',
 'Ability0_BuildCommandCenter',
 'Ability0_TrainMarine',
 'Ability0_UpgradeToOrbitalCommand',
 'Ability0_BuildFactory',
 'Ability0_Attack',
 'Ability0_BuildBarracksReactor',
 'Ability0_ExtraSupplies',
 'Ability0_BuildFactoryTechLab',
 'Ability0_KD8Charge',
 'Ability0_SCVRepair',
 'Ability0_BuildSiegeTank',
 'Ability0_BuildStarport',
 'Ability0_BuildEngineeringBay',
 'Ability0_BuildWidowMine',
 'Ability0_SiegeMode',
 'Ability0_TrainMedivac',
 'Ability0_UpgradeTerranInfantryArmor1',
 'Ability0_CancelSlot',
 'Ability0_CalldownMULE',
 'Ability0_LiftBarracks',
 'Ability0_BuildBarracksTechLab',
 'Ability0_LiftStarport',
 'Ability0_LandBarracks',
 'Ability0_TrainProbe',
 'Ability0_BuildPylon',
 'Ability

In [47]:

# 예측
preds = []
for model in models:
    pred = model.predict_proba(x_test)[:, 1]
    preds.append(pred)
pred = np.mean(preds, axis=0)

submission = pd.read_csv(SubmissingFile, index_col=0)
#display( submission )
submission['winner'] = submission['winner'] + pred
print(OutputSubmissionFile)
submission.to_csv( OutputSubmissionFile )
display( submission )

imsee//submission.csv


,winner
game_id,
38872,0.553361
38873,0.477954
38874,0.483374
38875,0.421650
38876,0.483476
...,...
55654,0.548035
55655,0.425954
55656,0.708207


In [55]:
submission['winner'].min()

0.2242753505706787

In [56]:
submission['winner'].max()

0.7876037359237671

In [57]:
submission['winner'].describe()

count    16787.000000
mean         0.501055
std          0.103085
min          0.224275
25%          0.428473
50%          0.498543
75%          0.570608
max          0.787604
Name: winner, dtype: float64